In [2]:
from openmm import *
from openmm.app import *
from openmm.unit import *
from sys import stdout

In [2]:
pdbfile = PDBFile('data/prep_complex.pdb')

modeller = Modeller(pdbfile.topology, pdbfile.positions)

forcefield = ForceField("amber14-all.xml", "amber14/tip3p.xml")
modeller.addSolvent(forcefield, model="tip3p", padding=0.5 * nanometer, boxShape='dodecahedron')
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, constraints=HBonds)
temperature = 300 * kelvin
pressure = 1 * bar
integrator = LangevinMiddleIntegrator(temperature, 1 / picosecond, 2 * femtoseconds)
system.addForce(MonteCarloBarostat(pressure, temperature))
platform = Platform.getPlatformByName('CUDA')
simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy(maxIterations=500)
positions = simulation.context.getState(getPositions=True).getPositions()
with open("init_romb.pdb", "w") as f:
    PDBFile.writeFile(simulation.topology, positions, f)

In [12]:
#Or load existing minimized system
pdbfile = PDBFile("init_romb.pdb")

modeller = Modeller(pdbfile.topology, pdbfile.positions)

forcefield = ForceField("amber14-all.xml", "amber14/tip3pfb.xml")
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, constraints=HBonds)
temperature = 300 * kelvin
pressure = 1 * bar
integrator = LangevinIntegrator(temperature, 1 / picosecond, 2 * femtoseconds)
system.addForce(MonteCarloBarostat(pressure, temperature))
platform = Platform.getPlatformByName('CUDA')
simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)

In [ ]:
simulation.reporters = []
simulation.reporters.append(DCDReporter("traj.dcd", 500))
simulation.reporters.append(
    StateDataReporter(stdout, 100, step=True, temperature=True, elapsedTime=True)
)
simulation.reporters.append(
    StateDataReporter(
        "scalars.csv",
        100,
        step=True,
        time=True,
        potentialEnergy=True,
        temperature=True,
        volume=True,
        density=True
    )
)
simulation.step(5000)

# The last line is only needed for Windows users,
# to close the DCD file before it can be opened by nglview.
del simulation

In [8]:
pdbfile = PDBFile('data/prep_complex.pdb')

modeller = Modeller(pdbfile.topology, pdbfile.positions)

forcefield = ForceField("amber14-all.xml", "implicit/gbn2.xml")
system = forcefield.createSystem(modeller.topology, nonbondedMethod=CutoffNonPeriodic, nonbondedCutoff=1.5*nanometer, constraints=HBonds,
                                 soluteDielectric=1.0, solventDielectric=80.0, implicitSolventKappa=1.0/nanometer)
temperature = 300 * kelvin
pressure = 1 * bar
integrator = LangevinMiddleIntegrator(temperature, 1 / picosecond, 2 * femtoseconds)
platform = Platform.getPlatformByName('CUDA')
simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy(maxIterations=500)
positions = simulation.context.getState(getPositions=True).getPositions()
with open("init_impl.pdb", "w") as f:
    PDBFile.writeFile(simulation.topology, positions, f)

In [9]:
simulation.reporters = []
simulation.reporters.append(DCDReporter("traj.dcd", 100))
simulation.reporters.append(
    StateDataReporter(stdout, 100, step=True, temperature=True, elapsedTime=True)
)
simulation.reporters.append(
    StateDataReporter(
        "scalars.csv",
        10,
        step=True,
        time=True,
        potentialEnergy=True,
        totalEnergy=True,
        temperature=True
    )
)
simulation.step(5000)

# The last line is only needed for Windows users,
# to close the DCD file before it can be opened by nglview.
del simulation

#"Step","Temperature (K)","Elapsed Time (s)"
100,56.37018772956284,8.082389831542969e-05


200,102.3204653319351,0.07141947746276855
300,138.21905654599377,0.1212301254272461
400,165.2421991752736,0.17021417617797852
500,193.8492723064342,0.21898651123046875
600,214.74408767920517,0.2686328887939453
700,232.3063962340731,0.30188965797424316
800,253.20321817715202,0.33396339416503906
900,255.39946899610004,0.36577796936035156
1000,264.04980082070034,0.3986690044403076
1100,263.7416784144228,0.4310016632080078
1200,273.72355194212275,0.46372318267822266
1300,278.9959084002436,0.496490478515625
1400,291.4521856718307,0.5289821624755859
1500,289.23275262560634,0.5613107681274414
1600,300.1397332646801,0.5934553146362305
1700,294.5223318297718,0.6260948181152344
1800,296.4713112734847,0.6581456661224365
1900,297.01609400020124,0.6901516914367676
2000,300.8719961557427,0.7225751876831055
2100,304.79935201842216,0.754317045211792
2200,310.9497338384184,0.7863674163818359
2300,296.4349477505997,0.8193507194519043
2400,306.5318702071416,0.8519718647003174
2500,303.160570929822,0.8847